# Modeling

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, RepeatedKFold
from sklearn.metrics import r2_score, mean_squared_error as mse, mean_absolute_error as mae, SCORERS
from sklearn.dummy import DummyRegressor
from catboost import Pool, CatBoostRegressor, cv
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from hyperopt import fmin, tpe, hp, Trials

In [ ]:
#load dataframe REGULAR
#parse datetime column
#df=pd.read_csv('COVID19_modeling.csv', parse_dates=[0])
#df.set_index('date', inplace= True)
#df.drop(columns='Unnamed: 0', inplace=True)

In [2]:
#load dataframe OHE
#parse datetime column
df_ohe = pd.read_csv('ohe_data.csv', parse_dates=['date'])
df_ohe.drop(columns='Unnamed: 0', inplace=True)
df_ohe.set_index('date', inplace=True)

In [3]:
df_ohe.head()

,Avg_Temp(F),Conf_Cases,day_of_week,day_of_year,Year,Month,Day,new_case_percent_pop*,state_id_CT,state_id_MA,state_id_ME,state_id_VT
date,,,,,,,,,,,,
2020-03-01,26.42,1.0,6,61,2020,3,1,0.014225,0,1,0,0
2020-03-02,36.50,1.0,0,62,2020,3,2,0.014225,0,1,0,0
2020-03-03,55.94,1.0,1,63,2020,3,3,0.014225,0,1,0,0
2020-03-04,46.94,2.0,2,64,2020,3,4,0.028450,0,1,0,0
2020-03-05,42.98,8.0,3,65,2020,3,5,0.113799,0,1,0,0


## Train-test split

Before training any models on the data, let's do a train-test split to keep training and testing data consistent and separated.

In [ ]:
# Create features
#X, y = df_ohe.drop(columns=['Conf_Cases', 'new_case_percent_pop*']), df_ohe['new_case_percent_pop*']
# Make test and train split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [4]:
len(df_ohe)

2224

In [5]:
bins = np.linspace(0, 2224, 200)

In [6]:
# Create features
X, y = df_ohe.drop(columns=['Conf_Cases', 'new_case_percent_pop*']), df_ohe['new_case_percent_pop*']

In [7]:
y_binned = np.digitize(y, bins)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y_binned, random_state=42)

In [ ]:
# Make test and train split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify =y_binned, random_state=42)

## Using the mean as a baseline prediction model

Previously, we determined the R2 score of using the mean to predict COVID19 cases for each individual state. Let's do the same thing now that we have all the states in one DataFrame, so that we will have a baseline "dummy" model to compare our future optimized models to. 

In [9]:
dummy_mean = DummyRegressor()
# "Train" dummy regressor
dummy_mean.fit(X_train, y_train)
# Get R2 score
score_dummy = dummy_mean.score(X_test, y_test)
print("The R2 score of using the mean to predict COVID19 cases in our states is:", score_dummy)

The R2 score of using the mean to predict COVID19 cases in our states is: -0.0008611149015183006


Let's store the evaluation metric values for the dummy regressor so we can compare them with our future models. 

In [10]:
dummy_pred = dummy_mean.predict(X_test)
dummy_r2 = r2_score(y_test, dummy_pred)
dummy_mse = mse(y_test, dummy_pred)
dummy_rmse = np.sqrt(mse(y_test, dummy_pred))

## Tuning the top performing models for ensemble model

In the pre-processing step, we determined (with the help of Pycaret) that our top performing models were **CatBoost Regressor**, **Random Forest Regressor**, and **Extra Trees Regressor**. Let's now fine tune the hyperparameters of each of these models, in preparation for feeding them into the pipeline of the Voting Regressor. 

**NOTE TO SELF: Determine hyperparameters for each model, but pass unfitted (tuned) models to the VotingRegressor**


### 1. Random Forest Regressor
With a few exceptions, a RandomForestClassifier has all the hyperparameters of a DecisionTreeClassifier (to control how trees are grown), plus all the hyperparameters of a BaggingClassifier to control the ensemble itself. I'll explore a couple different ways to determine optimal hyperparameters and will choose those that produce the best R2 and MSE for the final ensemble model (a VotingRegressor).

In [11]:
# Define parameters to search for GridSearchCV
basic_param_grid = {'n_estimators': [100, 300, 500, 900, 1200],
              'max_depth': [3, 5, 20, 50, 100],
              }
# Instantiate RandomForestRegressor
basic_rf = RandomForestRegressor(random_state=42)
cv_rf = GridSearchCV(basic_rf, basic_param_grid, cv = 5)
cv_rf_fit = cv_rf.fit(X_train, y_train)

In [12]:
print('The optimal max_depth for the RandomForestRegressor is: {}'.format(cv_rf_fit.best_params_['max_depth']))
print('The optimal n_estimators for the RandomForestRegressor is: {}'.format(cv_rf_fit.best_params_['n_estimators']))

The optimal max_depth for the RandomForestRegressor is: 50
The optimal n_estimators for the RandomForestRegressor is: 300


In [13]:
# Instantiate RFR with optimal hyperparameters
basic_rf = RandomForestRegressor(max_depth = 50, n_estimators = 300, random_state=42)
basic_rf.fit(X_train, y_train)
basic_rf_pred = basic_rf.predict(X_test)

In [14]:
# Calculate evaluation metrics on basic_rf
basic_rf_r2 = r2_score(y_test, basic_rf_pred)
basic_rf_mse = mse(y_test, basic_rf_pred)
basic_rf_rmse = np.sqrt(basic_rf_mse)

In [15]:
rf_results = pd.DataFrame({'Model':['dummy_reg', 'basic_rf'], 'R2': [dummy_r2, basic_rf_r2], 'MSE':[dummy_mse, basic_rf_mse], 'RMSE':[dummy_rmse, basic_rf_rmse]})

In [16]:
rf_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_rf,0.849384,42.810904,6.543004


***

#### Using RandomSearchCV to find optimal hyperparameter values

In [17]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(20, 120, num = 11)]
max_depth.append(None)
#max_depth.append(9)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [18]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, n_jobs=-1, random_state=42)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   44.5s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.3min finished


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [20, 30, 40, 50, 60, 70,
                                                      80, 90, 100, 110, 120,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4, 10],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, verbose=2)

In [19]:
rf_random.best_params_

{'n_estimators': 900,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': True}

In [20]:
# Instantiate RFR with optimal hyperparameters
rf2 = RandomForestRegressor(n_estimators= 900,
                             min_samples_split= 2,
                             min_samples_leaf= 2,
                             max_features= 'auto',
                             max_depth= 90,
                             bootstrap= True, 
                             random_state=42)
rf2.fit(X_train, y_train)
rf_pred2 = rf2.predict(X_test)

In [21]:
rf_random_r2 = r2_score(y_test, rf_pred2)
rf_random_mse = mse(y_test, rf_pred2)
rf_random_rmse = np.sqrt(mse(y_test, rf_pred2))

In [22]:
rf_results.loc[len(rf_results)]= ['rf_randomCV', 
                                 r2_score(y_test, rf_pred2),
                                 mse(y_test, rf_pred2),
                                 np.sqrt(mse(y_test, rf_pred2))]

In [23]:
rf_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_rf,0.849384,42.810904,6.543004
2,rf_randomCV,0.849289,42.838057,6.545079


***

### Using GridSearchCV to find the optimal hyperparameter values

In [25]:
# Create the parameter grid based on the results of random search 
GS_param_grid = {
    'bootstrap': [True],
    'max_depth': [70, 80, 90, 100, 110],
    'max_features': ['auto'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 3],
    'n_estimators': [700, 800, 900, 1000, 1100]
}

In [26]:
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = GS_param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [27]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  7.6min finished


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True],
                         'max_depth': [70, 80, 90, 100, 110],
                         'max_features': ['auto'],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3],
                         'n_estimators': [700, 800, 900, 1000, 1100]},
             verbose=2)

In [28]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 90,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 800}

In [29]:
# Instantiate RFR with optimal hyperparameters
rf3 = RandomForestRegressor(n_estimators= 800,
                             min_samples_split= 3,
                             min_samples_leaf= 2,
                             max_features= 'auto',
                             max_depth= 90,
                             bootstrap= True)
rf3.fit(X_train, y_train)
rf_pred3 = rf3.predict(X_test)

In [30]:
# Determine R2 score
print('The R2 score for the RandomForestRegressor with GSCV tuning is {}'.format(r2_score(y_test, rf_pred3)))

The R2 score for the RandomForestRegressor with GSCV tuning is 0.848008873583171


In [31]:
print('The MSE of the RandomForestRegressor with GSCV tuning is {}'.format(mse(y_test, rf_pred3)))

The MSE of the RandomForestRegressor with GSCV tuning is 43.201828280968115


In [32]:
rf_results.loc[len(rf_results)]= ['rf_gridCV', 
                                 r2_score(y_test, rf_pred3),
                                 mse(y_test, rf_pred3),
                                 np.sqrt(mse(y_test, rf_pred3))]

In [33]:
rf_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_rf,0.849384,42.810904,6.543004
2,rf_randomCV,0.849289,42.838057,6.545079
3,rf_gridCV,0.848009,43.201828,6.572810


### Bayesian Optimization with hyperopt

In [ ]:
#pip install hyperopt

In [46]:
trials = Trials()

In [47]:
space = {
    "n_estimators": hp.choice("n_estimators", [400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400]),
    "max_depth": hp.quniform("max_depth", 30, 120, 10),
}

In [48]:
# define objective function

def hyperparameter_tuning(params):
    rf_reg = RandomForestRegressor(**params,n_jobs=-1)
    r2 = cross_val_score(rf_reg, X_train, y_train ,scoring="r2").mean()
    return -r2

In [49]:
best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=100, 
    trials=trials
)

print("Best: {}".format(best))

100%|██████████| 100/100 [07:43<00:00,  4.64s/trial, best loss: -0.828803081186868]
Best: {'max_depth': 80.0, 'n_estimators': 0}


In [62]:
# Instantiate RFR with optimal hyperparameters
rf4 = RandomForestRegressor(n_estimators= 400,
                             max_depth= 80)
                             
rf4.fit(X_train, y_train)
rf_pred4 = rf4.predict(X_test)

In [63]:
rf_results.loc[len(rf_results)]= ['rf_bayes_hyperopt', 
                                 r2_score(y_test, rf_pred4),
                                 mse(y_test, rf_pred4),
                                 np.sqrt(mse(y_test, rf_pred4))]

In [65]:
rf_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_rf,0.849384,42.810904,6.543004
2,rf_randomCV,0.849289,42.838057,6.545079
3,rf_gridCV,0.848009,43.201828,6.572810
4,rf_bayes_hyperopt,0.846157,43.728343,6.612741


**?** $\star$ It looks like the optimal hyperparameters chosen by RandomSearchCV performed the best, both in terms of the R2 and mean squared error, so we'll use those hyperparameters (as defined in `rf2`) in our final Voting Regressor ensemble.

***

## 2. Extra Trees Regressor

### Using basic hyperparameter tuning only

In [67]:
# Define parameters to search for GridSearchCV
param_grid = {'n_estimators': [100, 300, 500, 900, 1200],
              'max_depth': [3, 5, 20, 50, 100],
              }
# Instantiate ExtraTreesRegressor
et = ExtraTreesRegressor(random_state=42)
cv_et = GridSearchCV(et, param_grid, cv = 5)
cv_et_fit = cv_et.fit(X_train, y_train)

#Print the best parameter
print(cv_et_fit.best_params_)

{'max_depth': 50, 'n_estimators': 1200}


In [68]:
print('The optimal max_depth for the ExtraTreesRegressor is: {}'.format(cv_et_fit.best_params_['max_depth']))
print('The optimal n_estimators for the ExtraTreesRegressor is: {}'.format(cv_et_fit.best_params_['n_estimators']))

The optimal max_depth for the ExtraTreesRegressor is: 50
The optimal n_estimators for the ExtraTreesRegressor is: 1200


In [70]:
# Instantiate ETR with optimal hyperparameters
et_basic = RandomForestRegressor(max_depth = 50, n_estimators = 1200)
et_basic.fit(X_train, y_train)
et_basic_pred = et_basic.predict(X_test)

In [71]:
# Determine R2 score
print('The R2 score for the basic Extra Trees Regressor is {}'.format(r2_score(y_test, et_basic_pred)))

The R2 score for the basic Extra Trees Regressor is 0.8488068725260061


In [72]:
print('The MSE of the basic ExtraTreesRegressor is {}'.format(mse(y_test, et_basic_pred)))

The MSE of the basic ExtraTreesRegressor is 42.97500574132715


In [73]:
basic_et_r2 = r2_score(y_test, et_basic_pred)
basic_et_mse = mse(y_test, et_basic_pred)
basic_et_rmse = np.sqrt(mse(y_test, et_basic_pred))

In [74]:
et_results = pd.DataFrame({'Model':['dummy_reg', 'basic_et'], 'R2': [dummy_r2, basic_et_r2], 'MSE':[dummy_mse, basic_et_mse], 'RMSE':[dummy_rmse, basic_et_rmse]})

In [75]:
et_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_et,0.848807,42.975006,6.555532


***

### Using RandomSearchCV to find optimal hyperparameter values

In [76]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(30, 140, num = 12)]
max_depth.append(None)
#max_depth.append(9)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10]
# Method of selecting samples for training each tree
bootstrap = [True]

# Create the random grid
random_grid_et = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 10], 'bootstrap': [True, False]}


In [77]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
et = ExtraTreesRegressor(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
et2 = RandomizedSearchCV(estimator = et, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
et2.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   59.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.4min finished


RandomizedSearchCV(cv=5, estimator=ExtraTreesRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [20, 30, 40, 50, 60, 70,
                                                      80, 90, 100, 110, 120,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4, 10],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, verbose=2)

In [78]:
et2.best_params_

{'n_estimators': 300,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 120,
 'bootstrap': True}

In [79]:
# Instantiate RFR with optimal hyperparameters
et_random = ExtraTreesRegressor(n_estimators= 300,
                             min_samples_split= 2,
                             min_samples_leaf= 1,
                             max_features= 'auto',
                             max_depth= 120,
                             bootstrap= True)
et_random.fit(X_train, y_train)
et_random_pred = et_random.predict(X_test)

In [80]:
# Determine R2 score
print('The R2 score for the ExtraTreesRegressor with RSCV tuning is {}'.format(r2_score(y_test, et_random_pred)))

The R2 score for the ExtraTreesRegressor with RSCV tuning is 0.8552342779275657


In [81]:
print('The MSE of the ExtraTreesRegressor with RSCV tuning is {}'.format(mse(y_test, et_random_pred)))

The MSE of the ExtraTreesRegressor with RSCV tuning is 41.14808550593898


In [82]:
et_results.loc[len(et_results)]= ['et_randomCV', 
                                 r2_score(y_test, et_random_pred),
                                 mse(y_test, et_random_pred),
                                 np.sqrt(mse(y_test, et_random_pred))]

In [83]:
et_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_et,0.848807,42.975006,6.555532
2,et_randomCV,0.855234,41.148086,6.414677


#### Using GridSearchCV to find the optimal hyperparamter values

In [87]:
# Create the parameter grid based on the results of random search 
GS_param_grid_et = {
    'bootstrap': [True],
    'max_depth': np.linspace(40, 150, 12),
    'max_features': ['auto'],
    'min_samples_leaf': [2, 3],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [300, 500, 800, 1000, 1200, 1600]
}

In [88]:
# Create a base model
et = ExtraTreesRegressor()
# Instantiate the grid search model
et3 = GridSearchCV(estimator = et, param_grid = GS_param_grid_et, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [89]:
# Fit the grid search to the data
et3.fit(X_train, y_train)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   50.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed: 15.2min finished


GridSearchCV(cv=5, estimator=ExtraTreesRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True],
                         'max_depth': array([ 40.,  50.,  60.,  70.,  80.,  90., 100., 110., 120., 130., 140.,
       150.]),
                         'max_features': ['auto'], 'min_samples_leaf': [2, 3],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [300, 500, 800, 1000, 1200, 1600]},
             verbose=2)

In [90]:
et3.best_params_

{'bootstrap': True,
 'max_depth': 70.0,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 500}

In [91]:
# Instantiate RFR with optimal hyperparameters
et_grid = ExtraTreesRegressor(n_estimators= 500,
                             min_samples_split= 4,
                             min_samples_leaf= 2,
                             max_features= 'auto',
                             max_depth= 70,
                             bootstrap= True)
et_grid.fit(X_train, y_train)
et_grid_pred = et_grid.predict(X_test)

In [92]:
# Determine R2 score
print('The R2 score for the Extra Trees Regressor with GSCV is {}'.format(r2_score(y_test, et_grid_pred)))

The R2 score for the Extra Trees Regressor with GSCV is 0.8547673410595482


In [93]:
print('The MSE of the ExtraTreesRegressor with GSCV is {}'.format(mse(y_test, et_grid_pred)))

The MSE of the ExtraTreesRegressor with GSCV is 41.28080724348845


In [94]:
et_results.loc[len(et_results)]= ['et_gridCV', 
                                 r2_score(y_test, et_grid_pred),
                                 mse(y_test, et_grid_pred),
                                 np.sqrt(mse(y_test, et_grid_pred))]

In [95]:
et_results 

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_et,0.848807,42.975006,6.555532
2,et_randomCV,0.855234,41.148086,6.414677
3,et_gridCV,0.854767,41.280807,6.425014


### Bayestian Optimization with hyperopt

In [96]:
trials_et = Trials()

In [97]:
space_et = {
    "n_estimators": hp.choice("n_estimators", [400, 500, 600, 700, 800, 900, 1000, 1100, 1200]),
    "max_depth": hp.quniform("max_depth", 30, 120, 10),
}

In [98]:
best_et = fmin(
    fn=hyperparameter_tuning,
    space = space_et, 
    algo=tpe.suggest, 
    max_evals=100, 
    trials=trials_et
)

print("Best: {}".format(best))

100%|██████████| 100/100 [07:05<00:00,  4.25s/trial, best loss: -0.8289119804924983]
Best: {'max_depth': 80.0, 'n_estimators': 0}


In [99]:
# Instantiate RFR with optimal hyperparameters
et4 = ExtraTreesRegressor(n_estimators= 400,
                             max_depth= 80)
                             
et4.fit(X_train, y_train)
et_pred4 = et4.predict(X_test)

In [100]:
et_results.loc[len(rf_results)]= ['et_bayes_hyperopt', 
                                 r2_score(y_test, et_pred4),
                                 mse(y_test, et_pred4),
                                 np.sqrt(mse(y_test, et_pred4))]

In [101]:
et_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_et,0.848807,42.975006,6.555532
2,et_randomCV,0.855234,41.148086,6.414677
3,et_gridCV,0.854767,41.280807,6.425014
5,et_bayes_hyperopt,0.853963,41.509520,6.442788


It seems that here the parameters suggested by RandomizedSearchCV (stored in `et_randomCV`) produced the best MSE and R2, so we'll use the hyperparameters from `et_randomCV`.

## 3. CatBoost Regressor

CatBoost is an open-sourced gradient boosting library. 
* 1. run CatBoost without tuning any hyperparameters to get baseline mse and r2
* 2. RandomizedSearchCV for hyperparameters
* 3. GridSearchCV for hyperparameters

* [Pool documentation](https://catboost.ai/en/docs/concepts/python-reference_pool)
* [CatboostRegressor documentation](https://catboost.ai/en/docs/concepts/python-reference_catboostregressor)

### WITHOUT POOL

In [134]:
cat_features_names = ['Month', 'day_of_week', 'state_id_MA', 'state_id_CT', 'state_id_VT', 'state_id_ME'] # here we specify names of categorical features
cat_features = [X.columns.get_loc(col) for col in cat_features_names]
print(cat_features)

[4, 1, 7, 6, 9, 8]


In [137]:
params = {'loss_function':'RMSE',
          'eval_metric':'R2',
          'cat_features': cat_features,
          'verbose': 200,
          'early_stopping_rounds': 200,
          'random_seed': 42
         }
cbc_1 = CatBoostRegressor(**params)
cbc_1.fit(X_train, y_train,
          eval_set=(X_test, y_test),
          use_best_model=True,
          plot=True
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.051377
0:	learn: 0.0585582	test: 0.0538505	best: 0.0538505 (0)	total: 2.16ms	remaining: 2.16s
200:	learn: 0.8833692	test: 0.8078675	best: 0.8080908 (195)	total: 404ms	remaining: 1.61s
400:	learn: 0.9300152	test: 0.8238552	best: 0.8238552 (400)	total: 747ms	remaining: 1.12s
600:	learn: 0.9510158	test: 0.8328266	best: 0.8328266 (600)	total: 1.17s	remaining: 776ms
800:	learn: 0.9616362	test: 0.8389936	best: 0.8390228 (799)	total: 1.57s	remaining: 390ms
999:	learn: 0.9687560	test: 0.8398203	best: 0.8399659 (995)	total: 1.97s	remaining: 0us

bestTest = 0.8399658723
bestIteration = 995

Shrink model to first 996 iterations.


In [218]:
pred_cbc1= cbc_1.predict(X_test)

In [220]:
r2_cbc1 = r2_score(y_test, pred_cbc1)
rmse_cbc1 = np.sqrt(mse(y_test, pred_cbc1))
mse_cbc1 = mse(y_test, pred_cbc1)

In [221]:
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse_cbc1))
print('MSE: {:.2f}'.format(mse_cbc1))
print('R2: {:.2f}'.format(r2_cbc1))

Testing performance
RMSE: 6.74
MSE: 45.49
R2: 0.84


### WITH POOL

In [222]:
train_dataset_cbc = Pool(X_train, y_train) 
test_dataset_cbc = Pool(X_test, y_test)

In [223]:
model_cbc2 = CatBoostRegressor(loss_function='RMSE', random_seed=42)

In [224]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

In [226]:
model_cbc2.grid_search(grid, train_dataset, cv)

0:	learn: 22.4931127	test: 21.5900708	best: 21.5900708 (0)	total: 1.1ms	remaining: 109ms
1:	learn: 22.1350946	test: 21.1984924	best: 21.1984924 (1)	total: 2.21ms	remaining: 108ms
2:	learn: 21.7857372	test: 20.8655757	best: 20.8655757 (2)	total: 3.16ms	remaining: 102ms
3:	learn: 21.4098002	test: 20.4461545	best: 20.4461545 (3)	total: 4.13ms	remaining: 99.1ms
4:	learn: 21.0396488	test: 20.0694328	best: 20.0694328 (4)	total: 5.03ms	remaining: 95.5ms
5:	learn: 20.7141893	test: 19.7640495	best: 19.7640495 (5)	total: 6.09ms	remaining: 95.4ms
6:	learn: 20.3798576	test: 19.4069260	best: 19.4069260 (6)	total: 6.89ms	remaining: 91.6ms
7:	learn: 20.0637578	test: 19.0674017	best: 19.0674017 (7)	total: 8.04ms	remaining: 92.4ms
8:	learn: 19.7503829	test: 18.7429287	best: 18.7429287 (8)	total: 9.06ms	remaining: 91.6ms
9:	learn: 19.4769032	test: 18.5138365	best: 18.5138365 (9)	total: 10.4ms	remaining: 93.2ms
10:	learn: 19.2291680	test: 18.2758318	best: 18.2758318 (10)	total: 11.5ms	remaining: 93.4ms
1

37:	learn: 11.0271730	test: 10.4389526	best: 10.4389526 (37)	total: 14.8ms	remaining: 24.1ms
38:	learn: 10.9729073	test: 10.4152886	best: 10.4152886 (38)	total: 15.5ms	remaining: 24.2ms
39:	learn: 10.9142352	test: 10.3310887	best: 10.3310887 (39)	total: 16.3ms	remaining: 24.4ms
40:	learn: 10.8592783	test: 10.2734764	best: 10.2734764 (40)	total: 17.2ms	remaining: 24.7ms
41:	learn: 10.7853206	test: 10.2495704	best: 10.2495704 (41)	total: 18.2ms	remaining: 25.1ms
42:	learn: 10.7423472	test: 10.2368211	best: 10.2368211 (42)	total: 19ms	remaining: 25.2ms
43:	learn: 10.7079711	test: 10.1980743	best: 10.1980743 (43)	total: 19.9ms	remaining: 25.3ms
44:	learn: 10.6707354	test: 10.1610217	best: 10.1610217 (44)	total: 20.6ms	remaining: 25.2ms
45:	learn: 10.6194186	test: 10.0949687	best: 10.0949687 (45)	total: 21.5ms	remaining: 25.3ms
46:	learn: 10.5745209	test: 10.0566905	best: 10.0566905 (46)	total: 22.3ms	remaining: 25.1ms
47:	learn: 10.5307766	test: 10.0198502	best: 10.0198502 (47)	total: 23.1

37:	learn: 14.7293380	test: 13.8442732	best: 13.8442732 (37)	total: 14.8ms	remaining: 24.2ms
38:	learn: 14.6357339	test: 13.7660638	best: 13.7660638 (38)	total: 15.4ms	remaining: 24ms
39:	learn: 14.5412182	test: 13.6685359	best: 13.6685359 (39)	total: 16.1ms	remaining: 24.2ms
40:	learn: 14.4485981	test: 13.5688230	best: 13.5688230 (40)	total: 17.2ms	remaining: 24.8ms
41:	learn: 14.3540395	test: 13.4589570	best: 13.4589570 (41)	total: 18.1ms	remaining: 25.1ms
42:	learn: 14.2698848	test: 13.3745218	best: 13.3745218 (42)	total: 19ms	remaining: 25.2ms
43:	learn: 14.1943955	test: 13.3134388	best: 13.3134388 (43)	total: 19.8ms	remaining: 25.2ms
44:	learn: 14.1159779	test: 13.2631267	best: 13.2631267 (44)	total: 20.7ms	remaining: 25.3ms
45:	learn: 14.0421072	test: 13.2033545	best: 13.2033545 (45)	total: 21.4ms	remaining: 25.2ms
46:	learn: 13.9668120	test: 13.1309159	best: 13.1309159 (46)	total: 22.2ms	remaining: 25ms
47:	learn: 13.8880422	test: 13.0505891	best: 13.0505891 (47)	total: 23.1ms	r

80:	learn: 9.2908295	test: 9.1490991	best: 9.1490991 (80)	total: 38.8ms	remaining: 33.1ms
81:	learn: 9.2437678	test: 9.1273354	best: 9.1273354 (81)	total: 39.3ms	remaining: 32.6ms
82:	learn: 9.1983718	test: 9.0762675	best: 9.0762675 (82)	total: 39.9ms	remaining: 32.2ms
83:	learn: 9.1782719	test: 9.0511103	best: 9.0511103 (83)	total: 40.9ms	remaining: 32.1ms
84:	learn: 9.1411178	test: 9.0034691	best: 9.0034691 (84)	total: 41.5ms	remaining: 31.7ms
85:	learn: 9.1134040	test: 8.9751683	best: 8.9751683 (85)	total: 41.9ms	remaining: 31.2ms
86:	learn: 9.0934806	test: 8.9576090	best: 8.9576090 (86)	total: 42.3ms	remaining: 30.6ms
87:	learn: 9.0728898	test: 8.9409323	best: 8.9409323 (87)	total: 42.7ms	remaining: 30.1ms
88:	learn: 9.0410584	test: 8.9110439	best: 8.9110439 (88)	total: 43.1ms	remaining: 29.5ms
89:	learn: 9.0184170	test: 8.8825646	best: 8.8825646 (89)	total: 43.5ms	remaining: 29ms
90:	learn: 8.9975541	test: 8.8470989	best: 8.8470989 (90)	total: 43.9ms	remaining: 28.5ms
91:	learn: 8

135:	learn: 8.2150657	test: 8.3672790	best: 8.3672790 (135)	total: 62.5ms	remaining: 6.43ms
136:	learn: 8.1973254	test: 8.3708512	best: 8.3672790 (135)	total: 63.5ms	remaining: 6.03ms
137:	learn: 8.1848559	test: 8.3631032	best: 8.3631032 (137)	total: 64.6ms	remaining: 5.62ms
138:	learn: 8.1717651	test: 8.3601961	best: 8.3601961 (138)	total: 65.1ms	remaining: 5.15ms
139:	learn: 8.1580090	test: 8.3553725	best: 8.3553725 (139)	total: 65.5ms	remaining: 4.68ms
140:	learn: 8.1505246	test: 8.3606760	best: 8.3553725 (139)	total: 65.9ms	remaining: 4.21ms
141:	learn: 8.1376202	test: 8.3549017	best: 8.3549017 (141)	total: 66.3ms	remaining: 3.73ms
142:	learn: 8.1306201	test: 8.3521102	best: 8.3521102 (142)	total: 66.7ms	remaining: 3.26ms
143:	learn: 8.1205089	test: 8.3391186	best: 8.3391186 (143)	total: 67ms	remaining: 2.79ms
144:	learn: 8.1139991	test: 8.3430043	best: 8.3391186 (143)	total: 67.4ms	remaining: 2.32ms
145:	learn: 8.0875386	test: 8.3141500	best: 8.3141500 (145)	total: 68ms	remaining:

...

51:	learn: 10.1910048	test: 9.8959970	best: 9.8959970 (51)	total: 80.5ms	remaining: 229ms
52:	learn: 10.0933468	test: 9.8187913	best: 9.8187913 (52)	total: 83.7ms	remaining: 232ms
53:	learn: 9.9843123	test: 9.7203416	best: 9.7203416 (53)	total: 86.3ms	remaining: 233ms
54:	learn: 9.8874098	test: 9.6490519	best: 9.6490519 (54)	total: 89.7ms	remaining: 236ms
55:	learn: 9.8059246	test: 9.5858914	best: 9.5858914 (55)	total: 93.1ms	remaining: 239ms
56:	learn: 9.7182540	test: 9.5399531	best: 9.5399531 (56)	total: 96.2ms	remaining: 241ms
57:	learn: 9.6336418	test: 9.4616091	best: 9.4616091 (57)	total: 99.4ms	remaining: 243ms
58:	learn: 9.5340618	test: 9.3771583	best: 9.3771583 (58)	total: 102ms	remaining: 245ms
59:	learn: 9.4441466	test: 9.2997928	best: 9.2997928 (59)	total: 105ms	remaining: 246ms
60:	learn: 9.3689581	test: 9.2381071	best: 9.2381071 (60)	total: 107ms	remaining: 244ms
61:	learn: 9.2754667	test: 9.1650343	best: 9.1650343 (61)	total: 109ms	remaining: 242ms
62:	learn: 9.1940317	te

192:	learn: 5.5507582	test: 6.7584633	best: 6.7584633 (192)	total: 462ms	remaining: 16.7ms
193:	learn: 5.5357452	test: 6.7440959	best: 6.7440959 (193)	total: 465ms	remaining: 14.4ms
194:	learn: 5.5279511	test: 6.7402464	best: 6.7402464 (194)	total: 469ms	remaining: 12ms
195:	learn: 5.5176079	test: 6.7328989	best: 6.7328989 (195)	total: 472ms	remaining: 9.64ms
196:	learn: 5.5056553	test: 6.7287419	best: 6.7287419 (196)	total: 476ms	remaining: 7.25ms
197:	learn: 5.4984662	test: 6.7262124	best: 6.7262124 (197)	total: 481ms	remaining: 4.86ms
198:	learn: 5.4882863	test: 6.7278560	best: 6.7262124 (197)	total: 484ms	remaining: 2.43ms
199:	learn: 5.4791451	test: 6.7257777	best: 6.7257777 (199)	total: 487ms	remaining: 0us

bestTest = 6.725777699
bestIteration = 199

94:	loss: 6.7257777	best: 6.1261168 (83)	total: 14.3s	remaining: 150ms
0:	learn: 21.2986362	test: 20.4468884	best: 20.4468884 (0)	total: 2.92ms	remaining: 581ms
1:	learn: 20.0924360	test: 19.3608246	best: 19.3608246 (1)	total: 6.28m

107:	learn: 4.2472333	test: 6.3901000	best: 6.3842698 (99)	total: 353ms	remaining: 301ms
108:	learn: 4.2296306	test: 6.3879756	best: 6.3842698 (99)	total: 356ms	remaining: 297ms
109:	learn: 4.2127076	test: 6.3684338	best: 6.3684338 (109)	total: 359ms	remaining: 293ms
110:	learn: 4.1834671	test: 6.3712270	best: 6.3684338 (109)	total: 361ms	remaining: 289ms
111:	learn: 4.1657105	test: 6.3664964	best: 6.3664964 (111)	total: 362ms	remaining: 285ms
112:	learn: 4.1415095	test: 6.3705898	best: 6.3664964 (111)	total: 364ms	remaining: 281ms
113:	learn: 4.1243967	test: 6.3738478	best: 6.3664964 (111)	total: 366ms	remaining: 276ms
114:	learn: 4.1058054	test: 6.3717068	best: 6.3664964 (111)	total: 368ms	remaining: 272ms
115:	learn: 4.0905590	test: 6.3727107	best: 6.3664964 (111)	total: 371ms	remaining: 269ms
116:	learn: 4.0710137	test: 6.3610224	best: 6.3610224 (116)	total: 374ms	remaining: 265ms
117:	learn: 4.0605806	test: 6.3607093	best: 6.3607093 (117)	total: 376ms	remaining: 261ms
118:	learn: 

0:	learn: 21.1365272	test: 21.1331367	best: 21.1331367 (0)
1:	learn: 19.6018757	test: 19.6336345	best: 19.6336345 (1)
2:	learn: 18.2511449	test: 18.3116974	best: 18.3116974 (2)
3:	learn: 17.0713118	test: 17.1764865	best: 17.1764865 (3)
4:	learn: 15.9336267	test: 16.0917074	best: 16.0917074 (4)
5:	learn: 14.9296454	test: 15.1783508	best: 15.1783508 (5)
6:	learn: 13.9872246	test: 14.2608299	best: 14.2608299 (6)
7:	learn: 13.3021475	test: 13.6076328	best: 13.6076328 (7)
8:	learn: 12.5963890	test: 12.9812372	best: 12.9812372 (8)
9:	learn: 11.9878759	test: 12.3991789	best: 12.3991789 (9)
10:	learn: 11.4326335	test: 11.8674962	best: 11.8674962 (10)
11:	learn: 10.9215345	test: 11.4350475	best: 11.4350475 (11)
12:	learn: 10.4242928	test: 10.9690550	best: 10.9690550 (12)
13:	learn: 10.0289772	test: 10.6313630	best: 10.6313630 (13)
14:	learn: 9.6488161	test: 10.3084279	best: 10.3084279 (14)
15:	learn: 9.2909102	test: 10.0305451	best: 10.0305451 (15)
16:	learn: 8.9708725	test: 9.7429510	best: 9.7

{'params': {'depth': 8,
  'iterations': 150,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.5},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               4

In [227]:
pred_cbc2 = model_cbc2.predict(X_test)
rmse_cbc2 = (np.sqrt(mse(y_test, pred_cbc2)))
r2_cbc2 = r2_score(y_test, pred_cbc2)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse_cbc2))
print('R2: {:.2f}'.format(r2_cbc2))

Testing performance
RMSE: 6.49
R2: 0.85


In [229]:
model_cbc2_params = {'depth': 8,
  'iterations': 150,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.5}

In [231]:
print(model_cbc2_params)

{'depth': 8, 'iterations': 150, 'learning_rate': 0.1, 'l2_leaf_reg': 0.5}


### WITH CV

In [237]:
model_cbc3 = CatBoostRegressor(random_seed=42)

In [240]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1, 0.3],
        'depth': [4, 6, 8, 10, 12],
        'l2_leaf_reg': [0.2, 0.5, 1, 3],
        'loss_function':'RMSE',
        'eval_metric':'R2',
        'cat_features': cat_features,
        }

In [241]:
model_cbc3.grid_search(grid, train_dataset)

CatBoostError: Parameter 'loss_function' currently is not supported in grid search

In [183]:
train_dataset = Pool(X_train, y_train) 
test_dataset = Pool(X_test, y_test)

In [188]:
params_cv = {'loss_function':'RMSE',
          'eval_metric':'R2',
          'cat_features': cat_features, 
          'early_stopping_rounds': 200,
          'verbose': 200,
          'random_seed': 42
         }
model_cv = CatBoostRegressor(**params)

In [189]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.001, 0.03, 0.1],
        'depth': [2, 4, 6, 8, 50, 100],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

In [190]:
res = model_cv.grid_search(grid, train_dataset)

0:	learn: -0.6159109	test: -0.6935221	best: -0.6935221 (0)	total: 1.32ms	remaining: 131ms
99:	learn: -0.4451989	test: -0.5054963	best: -0.5054963 (99)	total: 71.4ms	remaining: 0us

bestTest = -0.5054962527
bestIteration = 99

0:	loss: -0.5054963	best: -0.5054963 (0)	total: 81.4ms	remaining: 17.5s
0:	learn: -0.5591932	test: -0.6311567	best: -0.6311567 (0)	total: 440us	remaining: 43.6ms
99:	learn: 0.5861966	test: 0.5766514	best: 0.5766514 (99)	total: 43.2ms	remaining: 0us

bestTest = 0.5766513989
bestIteration = 99

1:	loss: 0.5766514	best: 0.5766514 (1)	total: 129ms	remaining: 13.8s
0:	learn: -0.4291612	test: -0.4881859	best: -0.4881859 (0)	total: 441us	remaining: 43.7ms
99:	learn: 0.7626938	test: 0.7321729	best: 0.7321729 (99)	total: 41.1ms	remaining: 0us

bestTest = 0.7321729128
bestIteration = 99

2:	loss: 0.7321729	best: 0.7321729 (2)	total: 175ms	remaining: 12.4s
0:	learn: -0.6159155	test: -0.6935268	best: -0.6935268 (0)	total: 439us	remaining: 43.5ms
99:	learn: -0.4453866	test: -0

199:	learn: 0.8218155	test: 0.7770987	best: 0.7770987 (199)	total: 92.4ms	remaining: 0us

bestTest = 0.7770986523
bestIteration = 199

29:	loss: 0.7770987	best: 0.7811415 (26)	total: 2.02s	remaining: 12.5s
0:	learn: -0.6159231	test: -0.6935346	best: -0.6935346 (0)	total: 429us	remaining: 85.4ms
199:	learn: -0.3030048	test: -0.3480476	best: -0.3480476 (199)	total: 87.4ms	remaining: 0us

bestTest = -0.3480475903
bestIteration = 199

30:	loss: -0.3480476	best: 0.7811415 (26)	total: 2.11s	remaining: 12.6s
0:	learn: -0.5595481	test: -0.6315185	best: -0.6315185 (0)	total: 800us	remaining: 159ms
199:	learn: 0.6901215	test: 0.6765978	best: 0.6765978 (199)	total: 94ms	remaining: 0us

bestTest = 0.6765978037
bestIteration = 199

31:	loss: 0.6765978	best: 0.7811415 (26)	total: 2.21s	remaining: 12.7s
0:	learn: -0.4302591	test: -0.4893013	best: -0.4893013 (0)	total: 1.01ms	remaining: 201ms
199:	learn: 0.8205038	test: 0.7759718	best: 0.7759718 (199)	total: 99.2ms	remaining: 0us

bestTest = 0.7759717

149:	learn: 0.7941481	test: 0.7678583	best: 0.7678583 (149)	total: 97.2ms	remaining: 0us

bestTest = 0.7678583301
bestIteration = 149

58:	loss: 0.7678583	best: 0.8418671 (50)	total: 4.58s	remaining: 12.2s
0:	learn: -0.4107516	test: -0.4766975	best: -0.4766975 (0)	total: 583us	remaining: 86.9ms
149:	learn: 0.8934853	test: 0.8385977	best: 0.8385977 (149)	total: 84.4ms	remaining: 0us

bestTest = 0.8385977441
bestIteration = 149

59:	loss: 0.8385977	best: 0.8418671 (50)	total: 4.67s	remaining: 12.1s
0:	learn: -0.6156050	test: -0.6933013	best: -0.6933013 (0)	total: 647us	remaining: 129ms
199:	learn: -0.2564066	test: -0.3019526	best: -0.3019526 (199)	total: 122ms	remaining: 0us

bestTest = -0.3019526445
bestIteration = 199

60:	loss: -0.3019526	best: 0.8418671 (50)	total: 4.79s	remaining: 12.2s
0:	learn: -0.5501461	test: -0.6246273	best: -0.6246273 (0)	total: 677us	remaining: 135ms
199:	learn: 0.8346608	test: 0.7949830	best: 0.7949830 (199)	total: 130ms	remaining: 0us

bestTest = 0.79498304

149:	learn: -0.3161653	test: -0.3697208	best: -0.3697208 (149)	total: 172ms	remaining: 0us

bestTest = -0.3697207942
bestIteration = 149

87:	loss: -0.3697208	best: 0.8508838 (62)	total: 8.09s	remaining: 11.8s
0:	learn: -0.5433729	test: -0.6150440	best: -0.6150440 (0)	total: 1.67ms	remaining: 249ms
149:	learn: 0.8664389	test: 0.8163355	best: 0.8163355 (149)	total: 131ms	remaining: 0us

bestTest = 0.8163354651
bestIteration = 149

88:	loss: 0.8163355	best: 0.8508838 (62)	total: 8.23s	remaining: 11.7s
0:	learn: -0.3781885	test: -0.4364836	best: -0.4364836 (0)	total: 1.18ms	remaining: 176ms
149:	learn: 0.9508443	test: 0.8582303	best: 0.8582925 (148)	total: 139ms	remaining: 0us

bestTest = 0.8582924697
bestIteration = 148

89:	loss: 0.8582925	best: 0.8582925 (89)	total: 8.38s	remaining: 11.7s
0:	learn: -0.6154103	test: -0.6930111	best: -0.6930111 (0)	total: 889us	remaining: 133ms
149:	learn: -0.3204262	test: -0.3749443	best: -0.3749443 (149)	total: 127ms	remaining: 0us

bestTest = -0.37494

99:	learn: 0.8513944	test: 0.8018442	best: 0.8018442 (99)	total: 162ms	remaining: 0us

bestTest = 0.8018441549
bestIteration = 99

115:	loss: 0.8018442	best: 0.8668342 (113)	total: 12.9s	remaining: 11.1s
0:	learn: -0.3822487	test: -0.4455221	best: -0.4455221 (0)	total: 1.37ms	remaining: 135ms
99:	learn: 0.9581758	test: 0.8598097	best: 0.8600617 (97)	total: 147ms	remaining: 0us

bestTest = 0.8600616507
bestIteration = 97

116:	loss: 0.8600617	best: 0.8668342 (113)	total: 13s	remaining: 11s
0:	learn: -0.6155947	test: -0.6932650	best: -0.6932650 (0)	total: 1.67ms	remaining: 166ms
99:	learn: -0.4198008	test: -0.4821248	best: -0.4821248 (99)	total: 143ms	remaining: 0us

bestTest = -0.4821247544
bestIteration = 99

117:	loss: -0.4821248	best: 0.8668342 (113)	total: 13.2s	remaining: 10.9s
0:	learn: -0.5497467	test: -0.6234874	best: -0.6234874 (0)	total: 1.75ms	remaining: 174ms
99:	learn: 0.8345101	test: 0.7912910	best: 0.7912910 (99)	total: 148ms	remaining: 0us

bestTest = 0.7912909972
bestIt

199:	learn: 0.9705093	test: 0.8662439	best: 0.8667095 (188)	total: 302ms	remaining: 0us

bestTest = 0.8667095339
bestIteration = 188

143:	loss: 0.8667095	best: 0.8686718 (125)	total: 20.1s	remaining: 10s
Estimating final quality...
0:	learn: -0.4124097	test: -0.4170057	best: -0.4170057 (0)
149:	learn: 0.9798298	test: 0.8499241	best: 0.8501153 (140)	total: 731ms	remaining: 0us


In [191]:
res

{'params': {'depth': 8,
  'iterations': 150,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.5},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               4

In [192]:
params_final = {'loss_function':'RMSE',
          'eval_metric':'R2',
          'cat_features': cat_features, 
          'early_stopping_rounds': 200,
          'verbose': 200,
          'random_seed': 42,
          'max_depth' : 8,
          'iterations' : 150,
          'learning_rate' : 0.1,
          'l2_leaf_reg': 0.5
         }

In [193]:
cb_model_tuned = CatBoostRegressor(**params)

In [208]:
cb_model_tuned.fit(X_train, y_train,
                   eval_set=(X_test, y_test))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.6537582	test: 0.5946772	best: 0.5946772 (0)	total: 1.11ms	remaining: 1.11s
200:	learn: 0.9959114	test: 0.7730314	best: 0.7899448 (17)	total: 163ms	remaining: 646ms
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7899448219
bestIteration = 17

Shrink model to first 18 iterations.


In [215]:
pred_z = cb_model_tuned.predict(X_test)

In [217]:
rmse_z = (np.sqrt(mse(y_test, pred_z)))
r2_z = r2_score(y_test, pred_z)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse_z))
print('R2: {:.2f}'.format(r2_z))

Testing performance
RMSE: 7.73
R2: 0.79


In [ ]:
pred = model.predict(X_test)
rmse = (np.sqrt(mse(y_test, pred)))
r2 = r2_score(y_test, pred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

In [203]:
y_test.dtypes

dtype('float64')

In [198]:
X_train.head()

,Avg_Temp(F),day_of_week,day_of_year,Year,Month,Day,state_id_CT,state_id_MA,state_id_ME,state_id_VT
date,,,,,,,,,,
2021-02-23,28.58,1,54,2021,2,23,0,0,1,0
2020-05-30,60.98,5,151,2020,5,30,0,0,1,0
2021-05-12,54.50,2,132,2021,5,12,0,1,0,0
2020-10-25,46.58,6,299,2020,10,25,1,0,0,0
2020-05-13,39.56,2,134,2020,5,13,0,0,1,0


### SOMETHING ELSE WITH CV

In [152]:
train_data = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_features
                 )

test_data = Pool(data=X_test,
                  label=y_test,
                  cat_features=cat_features
                 )

In [153]:
params = {'loss_function':'RMSE',
          'eval_metric':'R2',
#           'cat_features': cat_features, # we don't need to specify this parameter as 
#                                           pool object contains info about categorical features
          'early_stopping_rounds': 200,
          'verbose': 200,
          'random_seed': 42
         }

cbc_7 = CatBoostClassifier(**params)
cbc_7.fit(train_data, # instead of X_train, y_train
          eval_set=_data, # instead of (X_valid, y_valid)
          use_best_model=True, 
          plot=True
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.051377
0:	learn: 0.0585582	test: 0.0538505	best: 0.0538505 (0)	total: 1.98ms	remaining: 1.98s
200:	learn: 0.8833692	test: 0.8078675	best: 0.8080908 (195)	total: 378ms	remaining: 1.5s
400:	learn: 0.9300152	test: 0.8238552	best: 0.8238552 (400)	total: 741ms	remaining: 1.11s
600:	learn: 0.9510158	test: 0.8328266	best: 0.8328266 (600)	total: 1.18s	remaining: 782ms
800:	learn: 0.9616362	test: 0.8389936	best: 0.8390228 (799)	total: 1.56s	remaining: 388ms
999:	learn: 0.9687560	test: 0.8398203	best: 0.8399659 (995)	total: 1.97s	remaining: 0us

bestTest = 0.8399658723
bestIteration = 995

Shrink model to first 996 iterations.


In [182]:
#print(cbc_7.get_all_params())

In [243]:
params = {'loss_function':'RMSE',
          'eval_metric':'R2',
          'verbose': 200,
          'random_seed': 42
         }

all_train_data = Pool(data=X_train,
                      label=y_train,
                      cat_features=cat_features
                     )

scores = cv(
   params=params,
   dtrain=train_data,
   iterations=None,
   num_boost_round=None,
   fold_count=5,
   nfold=None,
   inverted=False,
   partition_random_seed=0,
   seed=42,
   shuffle=True,
   stratified=None,
   as_pandas=True,
   metric_period=None,
   verbose=None,
   verbose_eval=None,
   plot=True,
   early_stopping_rounds=200,
   folds=None,
   type='Classical',
   return_models=False)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: -0.5710208	test: -0.5758052	best: -0.5758052 (0)
200:	learn: 0.8388259	test: 0.7976584	best: 0.7976584 (200)
400:	learn: 0.8955728	test: 0.8241895	best: 0.8241895 (400)
600:	learn: 0.9228827	test: 0.8362699	best: 0.8362781 (598)
800:	learn: 0.9387816	test: 0.8418658	best: 0.8418658 (800)
999:	learn: 0.9497280	test: 0.8458919	best: 0.8458919 (999)	total: 15.9s	remaining: 0us


In [242]:
scores

,iterations,test-R2-mean,test-R2-std,train-R2-mean,train-R2-std,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0,-0.575805,0.053502,-0.571021,0.011212,22.299151,0.827847,22.304663,0.208433
1,1,-0.514693,0.051223,-0.509603,0.010396,21.862393,0.807690,21.864369,0.207964
2,2,-0.459743,0.050094,-0.454906,0.008814,21.462311,0.802721,21.464660,0.205016
3,3,-0.402779,0.045412,-0.398229,0.010612,21.039958,0.787967,21.042280,0.193007
4,4,-0.349068,0.042319,-0.343751,0.012778,20.633717,0.779429,20.628096,0.182472
...,...,...,...,...,...,...,...,...,...
995,995,0.845860,0.017512,0.949543,0.002296,6.960824,0.307220,3.996893,0.118029
996,996,0.845857,0.017547,0.949592,0.002285,6.960859,0.308064,3.994961,0.117612
997,997,0.845863,0.017487,0.949644,0.002289,6.960814,0.307631,3.992888,0.117725
998,998,0.845882,0.017524,0.949691,0.002275,6.960366,0.308724,3.991038,0.117369


In [160]:
cbc_7.get_feature_importance(prettified=True)

,Feature Id,Importances
0,Month,23.018774
1,day_of_year,22.450995
2,Year,10.938447
3,state_id_MA,8.566963
4,state_id_VT,8.193076
5,Avg_Temp(F),7.140689
6,state_id_CT,6.186531
7,day_of_week,5.209193
8,state_id_ME,4.490036
9,Day,3.805296


In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
df_ohe_features = df_ohe.drop(columns=['Conf_Cases', 'new_case_percent_pop*'])

In [ ]:
sorted_feature_importance = model.feature_importances_.argsort()
plt.barh(df_ohe_features.columns[sorted_feature_importance], 
        model.feature_importances_[sorted_feature_importance], 
        color='turquoise')
plt.xlabel('CatBoost Feature Importance')

## Shap value plot

Optimized RG model SHAP value plots:
    * Dot represents feature importance for one sample
    * Right $\Rightarrow$ feature increased probability of positive label
    * Left $\Rightarrow$ feature decreased probability of positivity label
    * Red $\Rightarrow$ high feature value
    * Blue $\Leftarrow$ low feature value

for more info on shap plots and how to interpret: [this article](https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d)

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test, feature_names = df_ohe_features.columns[sorted_feature_importance])

***

In [ ]:
#load dataframe OHE
#parse datetime column
df_cb = pd.read_csv('ohe_data2.csv', parse_dates=['date'])
df_cb.drop(columns='Unnamed: 0', inplace=True)
df_cb.set_index('date', inplace=True)

In [ ]:
df_cb.drop(columns=['Conf_Cases'], inplace=True)

In [ ]:
# Create features
X2, y2 = df_ohe.drop(columns=['new_case_percent_pop*']), df_ohe['new_case_percent_pop*']

In [ ]:
# Sort y
y_sorted2 = sorted(y2)

In [ ]:
# Save your y values in a new ndarray, broken down by the bins created above.
y_binned2 = np.digitize(y2, bins)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, stratify=y_binned2, random_state=42)

In [ ]:
train_dataset2 = Pool(X_train2, y_train2) 
test_dataset2 = Pool(X_test2, y_test2)

In [ ]:
ohe_all_model = CatBoostRegressor(loss_function='RMSE', random_seed=42)

In [ ]:
ohe_all_model.grid_search(grid, train_dataset2, cv=5)

In [ ]:
pred2 = ohe_all_model.predict(X_test2)
rmse = (np.sqrt(mse(y_test2, pred2)))
r2_2 = r2_score(y_test2, pred2)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

In [ ]:
sorted_feature_importance = ohe_all_model.feature_importances_.argsort()
plt.barh(df_cb.columns[sorted_feature_importance], 
        ohe_all_model.feature_importances_[sorted_feature_importance], 
        color='turquoise')
plt.xlabel('CatBoost Feature Importance')

In [ ]:
explainer2 = shap.TreeExplainer(model)
shap_values2 = explainer2.shap_values(X_test2)
shap.summary_plot(shap_values2, X_test2, feature_names = df_cb.columns[sorted_feature_importance])

## For catboost use: 'model'

****

#### Voting Regressor

In [ ]:
# RandomForestRegressor
rf_reg = RandomForestRegressor(bootstrap = True,
                             max_depth = 70,
                             max_features = 'auto',
                             min_samples_leaf = 2,
                             min_samples_split = 2,
                             n_estimators = 600
)

# ExtraTreesRegressor
et_reg = ExtraTreesRegressor(n_estimators = 1700,
                             min_samples_split = 3,
                             min_samples_leaf = 1,
                             max_features = 'auto',
                             max_depth = 70,
                             bootstrap = True)

# CatBoostRegressor
cb_reg = model

In [ ]:
# Create the Voting Classifier with soft voting because all models performed similarly

clf_voting = VotingRegressor(estimators = [('rf_est', rf_reg), ('et_est', et_reg),
                                           ('cb_est', cb_reg)])
#Fit and predict

clf_voting.fit(X_train, y_train)
pred_voting = clf_voting.predict(X_test)

In [ ]:
print('Unweighted VotingReg R2: ', r2_score(y_test, pred_voting))
print('Unweighted VotingReg MSE: ', mse(y_test, pred_voting))
print('Unweighted VotingReg RMSE: ', np.sqrt(mse(y_test, pred_voting)))

In [ ]:
# Create empty lists that will storage the different weights

weights1 = []
weights2 = []
weights3 = []
scores = []

# Create a for loop to evaluate different combinations of weights

for i in np.arange(0.1,1, 0.1):
    for j in np.arange(0.1,1, 0.1):
        for k in np.arange(0.1,1, 0.1):
            reg_voting = VotingRegressor(estimators = [('rf_est', rf_reg), ('et_est', et_reg),
                                           ('cb_est', cb_reg)], weights = [i, j, k])
            reg_voting.fit(X_train, y_train)
            pred = reg_voting.predict(X_test)
            score = r2_score(y_test, pred)
            scores.append(score)
            weights1.append(i)
            weights2.append(j)
            weights3.append(k)

In [ ]:
# Save the results in a data frame

test_scores = pd.DataFrame()
test_scores['Weight1'] = weights1
test_scores['Weight2'] = weights2
test_scores['Weight3'] = weights3
test_scores['Test Score'] = scores

# Create an additional column to save the sum of all the weights

test_scores['sum_weights'] = test_scores['Weight1'].add(test_scores['Weight2']).add(test_scores['Weight3'])

#We are only getting the rows that the sum of all weights were equal to one

condition = test_scores['sum_weights'] == 1

test_scores = test_scores.loc[condition]

In [ ]:
# Sort the values to see the different test scores depending on the weights
test_scores.sort_values(by = 'Test Score', ascending = False).head(15)

In [ ]:
# Create the Voting Classifier with the most equally weighted because all models performed similarly

reg_voting = VotingRegressor(estimators = [('rf_est', rf_reg), ('et_est', et_reg),
                                           ('cb_est', cb_reg)], weights = [0.2, 0.4, 0.4])

#Fit and predict

reg_voting.fit(X_train, y_train)
pred_voting = reg_voting.predict(X_test)

In [ ]:
R2_final = r2_score(y_test, pred_voting)
MSE_final = mse(y_test, pred_voting)
RMSE_final = np.sqrt(mse(y_test, pred_voting))

In [ ]:
print('Final VotingRegressor R2: ', R2_final)
print('Final VotingRegressor MSE: ', MSE_final)
print('Final VotingRegressor RMSE: ', RMSE_final)

## Feature importance

In [ ]:
X_train

In [ ]:
for i, item in enumerate(reg_voting.feature_importances_):
    print("{0:s}: {1:.2f}".format(X.columns[i], item))